In [307]:
import pandas as pd
import numpy as np
import random

import seaborn as sns
import matplotlib.pyplot as plt

from solver import Instance, Solver
from argparse import Namespace
from gurobipy import Model, GRB, tupledict
import json

import ast

In [1]:
city = "berlin"
demand_baseline = "1.00"
demand_type = "uniform"

demand_file = f'{city}_db={demand_baseline}_dt={demand_type}.csv'
print(demand_file)

berlin_db=1.00_dt=uniform.csv


In [309]:
demand_df = pd.read_csv(demand_file, index_col=0)

In [310]:
# PARAMETERS
# Regions
R = sorted(list(demand_df['region_id'].unique()))

region_area_map = {}
for r in R:
    for a in list(demand_df.query(f'region_id == {r}')['area_id'].unique()):
        region_area_map[a] = r
        
# Areas 
A = {}
area_map = {a: i for i, a in enumerate(list(demand_df['area_id'].unique())) }
areas = list(area_map.keys())
n_areas = len(areas)
for r in R:
    A[r] = [ area_map[a] for a in list(demand_df.query(f'region_id == {r}')['area_id'].unique()) ]

# Employees
n_employees = 12
employees = [e for e in range(n_employees)]
E = {}
#for r in R:
#    E[r] = employees
E[0] = [0,1,2]
E[1] = [3,4,5]
E[2] = [6,7,8]
E[3] = [9,10,11]
 
# Set of all shifts available
P = {}
n_shifts = 1
for r in R:
    P[r] = list(range(n_shifts))

# WE NEED TO SET starting and end time for each shift!! and ADD THE CONSTRAINTS so no one can work before or after this time!
# HOW??
shifts_start = {}

# Periods
n_periods = 8
Theta = [i for i in range(n_periods)]

# Days
n_days = 7
D = [d for d in range(n_days)]

# Set of demand scenarios
# TODO

# Hours in shift
h = {}
n_hours = 8
for p in range(n_shifts):
    h[p] = n_hours

# Cost outsource
c_out = 1.5

# Number of deliveries to perform (n_{a theta d})
deliveries = np.zeros((n_areas, n_periods, n_days))
for i, a in enumerate(areas):
    for k, d in enumerate(D):
        period_demands = demand_df.query(f'area_id == {a} & day == {d}')
        deliveries[i, :, k] = ast.literal_eval(period_demands['demand'].values[0])

# Number of couriers needed (m_{a theta d})
couriers_needed = np.zeros((n_areas, n_periods, n_days))
for i, a in enumerate(areas):
    for k, d in enumerate(D):
        period_demands = demand_df.query(f'area_id == {a} & day == {d}')
        couriers_needed[i, :, k] = ast.literal_eval(period_demands['required_couriers'].values[0])

# Cost of employed courier (c_{a theta d})
cost_couriers = np.zeros((n_areas, n_periods, n_days))
cost_courier = 1
for i, a in enumerate(areas):
    for j, t in enumerate(Theta):
        for k, d in enumerate(D):
            cost_couriers[i, j, k] = cost_courier

# Min hours worked for employee e
min_hours_worked = 8*3
h_min = {e: min_hours_worked for e in employees}

# Max hours worked for employee e
max_hours_worked = 8*6
h_max = {e: max_hours_worked for e in employees}

# Max differing starts
max_unique_starts = 2
b_max = {e: max_unique_starts for e in employees}

# mega_value
M = 9999

# DECISION VARIABLES

In [311]:
m = Model()

# r_{e p d}
r_var = m.addVars(n_employees, n_shifts, n_days, vtype=GRB.BINARY, name='r')

# k_{e a theta d}
k_var = m.addVars(n_employees, n_areas, n_periods, n_days, vtype=GRB.BINARY, name='k')

# U_{e p}
u_var = m.addVars(n_employees, n_shifts, vtype=GRB.BINARY, name='u')

# omega_{a theta d}
omega_var = m.addVars(n_areas, n_periods, n_days, vtype=GRB.CONTINUOUS, lb=0, name='omega')

# CONSTRAINTS

In [312]:
# 1. Connecting employees moving around areas to shift assignment p
for r in R:
    for e in E[r]:
        for p in P[r]:
            for d in D:
                Ar = A[r]
                m.addConstr(
                    (sum([k_var[e, a, theta, d] for a in Ar for theta in Theta]) == 1/2 * h[p] * r_var[e,p,d] ),
                    name = f'moving_areas_{r}_{e}_{p}_{d}'          
                )

In [313]:
# 2. Employee can only be assigned to one area at a time 
for r in R:
    for e in E[r]:
        for theta in Theta:
            for d in D:
                Ar = A[r]
                m.addConstr((sum([k_var[e, a, theta, d] for a in Ar]) <= 1),
                    name = f'employee_{r}_{e}_{theta}_{d}'          
                )

# ADDED: They only can be assigned one region!
#for e in employees:
#    for theta in Theta:
#        for d in D:
#            m.addConstr((sum([k_var[e, area_map[a], theta, d] for a in areas]) <= 1),
#                name = f'only_one_area_{e}_{theta}_{d}'          
#            )

In [314]:
# 3. Employee can only work one shift a day
for r in R:
    for es in E[r]:
        for d in D:
            Pr = P[r]
            m.addConstr((sum([r_var[e, p, d] for p in Pr]) <= 1),
                name = f'only_one_shift_{r}_{e}_{theta}_{d}'          
            )

In [315]:
# 4. One rest day a week
for r in R:
    for e in E[r]:
        Pr = P[r]
        m.addConstr((sum([r_var[e, p, d] for p in Pr for d in D]) <= 6),
            name = f'rest_day_{r}_{e}'          
        )

In [316]:
# 5. Min - Max hours worked per week
for r in R:
    for e in E[r]:
        min_hours = h_min[e]
        max_hours = h_max[e]

        m.addConstr(( h[p] * sum([r_var[e, p, d] for p in Pr for d in D]) >= min_hours),
            name = f'min_hours_{r}_{e}'          
        )

        m.addConstr(( h[p] * sum([r_var[e, p, d] for p in Pr for d in D]) <= max_hours),
            name = f'max_hours_{r}_{e}'          
        )

In [317]:
# 6. Different shifting times constraint
for r in R:
    for e in E[r]:
        for p in P[r]:
            m.addConstr((sum([r_var[e, p, d] for d in D]) <= u_var[e, p] * M),
                name = f'start_times_{r}_{e}_1'          
            )

for r in R:
    for e in E[r]:
        Pr = P[r]
        m.addConstr((sum([r_var[e, p, d] for p in Pr]) <= b_max[e]),
            name = f'start_times_{r}_{e}_2'          
        )

In [318]:
# 7. Employee can't work then outsource
for r in R:
    for a in A[r]:
        for theta in Theta:
            for d in D:
                # Be careful with parantheses!!
                if couriers_needed[a,theta,d] > 0:
                    factor = deliveries[a,theta,d] / couriers_needed[a,theta,d] * c_out 
                else:
                    factor = 0
                m.addConstr(
                    ((couriers_needed[a, theta, d] - sum([k_var[e, a, theta, d] for e in E[r]])) * factor <= omega_var[a, theta, d]),
                    name = f'outsource_{r}_{a}_{theta}_{d}'   
                )

# OBJECTIVE FUNCTION

In [319]:
factor_s = 1
m.setObjective(
    sum([
        sum([cost_couriers[a,theta,d] * k_var[e,a,theta,d] for e in E[r]]) 
            + omega_var[a,theta,d]
    for r in R for a in A[r] for theta in Theta for d in D]
    )
)

In [320]:
m.ModelSense = GRB.MINIMIZE
m.optimize()

Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[rosetta2])

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4204 rows, 43048 columns and 31879 nonzeros
Model fingerprint: 0xc1a02012
Variable types: 3304 continuous, 39744 integer (39744 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+01]
Found heuristic solution: objective 9600.7500000


Presolve removed 3081 rows and 39315 columns
Presolve time: 0.09s
Presolved: 1123 rows, 3733 columns, 9511 nonzeros
Found heuristic solution: objective 8525.2500000
Variable types: 0 continuous, 3733 integer (3727 binary)

Root relaxation: objective 8.210750e+03, 1564 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8210.75000    0   38 8525.25000 8210.75000  3.69%     -    0s
H    0     0                    8217.7500000 8210.75000  0.09%     -    0s
H    0     0                    8216.2500000 8210.75000  0.07%     -    0s
H    0     0                    8213.2500000 8210.75000  0.03%     -    0s

Cutting planes:
  Gomory: 3
  MIR: 2
  Zero half: 12

Explored 1 nodes (1859 simplex iterations) in 0.18 seconds (0.10 work units)
Thread count was 8 (of 8 available processors)

Solution count 5: 8213.25 8216.25 8217.75 ... 9600.75

O

# SOLUTION

In [321]:
# Total couriers in day
couriers_needed_ = []
for d in D:
    data = (
        pd.DataFrame(couriers_needed[:,:,d], index=areas, columns=Theta).reset_index()
        .rename(columns={'index': 'area'})
        #.assign(day = d)
    )

    area_day_data = (
        pd.melt(data, id_vars=['area'], value_vars=set(data.columns).difference('area'))
        .rename(columns={'variable': 'period', 'value': 'n_couriers'})
        .assign(day = d)
    )
    couriers_needed_.append(area_day_data)

couriers_needed_df = pd.concat(couriers_needed_)
couriers_needed_df

,area,period,n_couriers,day
0,10115,0,1.0,0
1,10117,0,0.0,0
2,10119,0,1.0,0
3,10178,0,1.0,0
4,10179,0,1.0,0
...,...,...,...,...
467,10965,7,1.0,6
468,10967,7,1.0,6
469,10969,7,1.0,6
470,10997,7,1.0,6


In [322]:
# Demand
deliveries_ = []
for d in D:
    data = (
        pd.DataFrame(deliveries[:,:,d], index=areas, columns=Theta).reset_index()
        .rename(columns={'index': 'area'})
        #.assign(day = d)
    )

    area_day_data = (
        pd.melt(data, id_vars=['area'], value_vars=set(data.columns).difference('area'))
        .rename(columns={'variable': 'period', 'value': 'deliveries'})
        .assign(day = d)
    )
    deliveries_.append(area_day_data)

deliveries_df = pd.concat(deliveries_)
deliveries_df

,area,period,deliveries,day
0,10115,0,3.0,0
1,10117,0,0.0,0
2,10119,0,2.0,0
3,10178,0,1.0,0
4,10179,0,1.0,0
...,...,...,...,...
467,10965,7,2.0,6
468,10967,7,2.0,6
469,10969,7,4.0,6
470,10997,7,5.0,6


In [323]:
couriers_needed_df = pd.merge(couriers_needed_df, deliveries_df, on=['area', 'period', 'day'])

In [324]:
# Employee shifts
employees_shifts = []
for e in employees:
    for p in range(n_shifts):
        for d in D:
            value = r_var[e, p, d].X
            if value > 0.5:
                hours_worked = h[p]
                employees_shifts.append([e, p, d, hours_worked])

employees_shifts_df = pd.DataFrame(employees_shifts, columns=['employee', 'shift', 'day', 'hours_worked'])
employees_shifts_df

,employee,shift,day,hours_worked
0,0,0,0,8
1,0,0,1,8
2,0,0,2,8
3,0,0,3,8
4,0,0,4,8
...,...,...,...,...
67,11,0,1,8
68,11,0,2,8
69,11,0,3,8
70,11,0,5,8


In [325]:
# Total hours worked
employees_shifts_df.groupby('employee')['hours_worked'].sum()

employee
0     48
1     48
2     48
3     48
4     48
5     48
6     48
7     48
8     48
9     48
10    48
11    48
Name: hours_worked, dtype: int64

In [326]:
# Area Employee assignment
area_employee_assignment = []
for e in employees:
    for a in areas:
        for theta in Theta:
            for d in D:
                value = k_var[e, area_map[a],theta,d].X
                if value > 0.5:
                    area_employee_assignment.append([e, a, theta, d])

area_employee_assignment_df = pd.DataFrame(area_employee_assignment, columns=['employee', 'area', 'period', 'day'])
area_employee_assignment_df

,employee,area,period,day
0,0,10247,0,3
1,0,10247,2,4
2,0,10247,3,0
3,0,10247,4,0
4,0,10247,4,2
...,...,...,...,...
283,11,10717,4,0
284,11,10717,7,1
285,11,10777,1,0
286,11,10777,1,3


In [327]:
(
    area_employee_assignment_df
    .query('employee == 0')
    .merge(couriers_needed_df, on=['day', 'area', 'period'])
    .assign(
        region = area_employee_assignment_df['area'].map(region_area_map)
    )
    .sort_values(['day', 'period'])
    [['employee', 'day', 'period', 'area', 'region', 'n_couriers', 'deliveries']]
    .head(100)
)

,employee,day,period,area,region,n_couriers,deliveries
2,0,0,3,10247,0,1.0,4.0
3,0,0,4,10247,0,1.0,5.0
10,0,0,5,10249,0,1.0,5.0
7,0,0,7,10247,0,1.0,5.0
20,0,1,0,10365,0,1.0,4.0
18,0,1,2,10318,0,1.0,5.0
5,0,1,5,10247,0,1.0,4.0
13,0,1,7,10249,0,1.0,5.0
4,0,2,4,10247,0,1.0,4.0
6,0,2,5,10247,0,1.0,5.0


In [328]:
(
    area_employee_assignment_df
    .query('employee == 1')
    #.merge(couriers_needed_df, on=['day', 'area', 'period'])
    .assign(
        region = area_employee_assignment_df['area'].map(region_area_map)
    )
    .sort_values(['day', 'period'])
    [['employee', 'day', 'period', 'area', 'region']] #, 'n_couriers']]
    .head(100)
)

,employee,day,period,area,region
24,1,0,0,10245,0
28,1,0,1,10247,0
31,1,0,2,10249,0
44,1,0,3,10365,0
25,1,1,0,10245,0
32,1,1,3,10249,0
47,1,1,5,10369,0
30,1,1,7,10247,0
42,1,2,3,10319,0
40,1,2,5,10318,0


In [329]:
# areas
area_period_days_df = (
    area_employee_assignment_df
    .groupby(['area', 'period', 'day'])
    .agg({'employee': ['count', 'unique']})
    .reset_index()
)
area_period_days_df.columns = ['area', 'period', 'day', 'employee_count', 'employees_assign']
area_period_days_df

,area,period,day,employee_count,employees_assign
0,10115,0,5,1,[4]
1,10115,2,6,1,[5]
2,10115,4,0,1,[5]
3,10115,5,2,1,[4]
4,10117,2,3,1,[3]
...,...,...,...,...,...
281,10999,6,3,1,[6]
282,10999,6,4,1,[7]
283,10999,6,6,1,[6]
284,10999,7,2,1,[7]


In [330]:
# Outsourcing
outsourcing_shifts = []
for a in areas:
    for theta in Theta:
        for d in D:
            value = omega_var[area_map[a],theta,d].X
            #print(f'{a} {theta} {d} : {value}')
            if value > 0.0:
                outsourcing_shifts.append([a, theta, d, value])

outsourcing_shifts_df = pd.DataFrame(outsourcing_shifts, columns=['area', 'period', 'day', 'cost_outsource'])
outsourcing_shifts_df

,area,period,day,cost_outsource
0,10115,0,0,4.5
1,10115,0,1,3.0
2,10115,0,2,1.5
3,10115,0,3,4.5
4,10115,0,4,9.0
...,...,...,...,...
2454,10999,6,5,4.5
2455,10999,7,1,6.0
2456,10999,7,4,6.0
2457,10999,7,5,4.5


In [331]:
# Join all
whole_solution_df = (
    couriers_needed_df
    # Employees
    .merge(area_period_days_df,  on=['area', 'period', 'day'], how='left')
    # Outsource
    .merge(outsourcing_shifts_df, on=['area', 'period', 'day'], how='left')
)
whole_solution_df.head(30)

,area,period,n_couriers,day,deliveries,employee_count,employees_assign,cost_outsource
0,10115,0,1.0,0,3.0,NaN,NaN,4.5
1,10117,0,0.0,0,0.0,NaN,NaN,NaN
2,10119,0,1.0,0,2.0,NaN,NaN,3.0
3,10178,0,1.0,0,1.0,NaN,NaN,1.5
4,10179,0,1.0,0,1.0,NaN,NaN,1.5
5,10243,0,1.0,0,5.0,1.0,[6],NaN
6,10245,0,1.0,0,5.0,1.0,[1],NaN
7,10247,0,1.0,0,3.0,NaN,NaN,4.5
8,10249,0,1.0,0,4.0,NaN,NaN,6.0
9,10315,0,1.0,0,2.0,NaN,NaN,3.0


In [332]:
whole_solution_df.query('n_couriers == 2')

,area,period,n_couriers,day,deliveries,employee_count,employees_assign,cost_outsource
85,10559,1,2.0,0,7.0,1.0,[9],5.25
115,10969,1,2.0,0,6.0,NaN,NaN,9.00
176,10999,2,2.0,0,7.0,NaN,NaN,10.50
182,10243,3,2.0,0,8.0,1.0,[6],6.00
236,10115,4,2.0,0,8.0,1.0,[5],6.00
...,...,...,...,...,...,...,...,...
3193,10247,6,2.0,6,6.0,NaN,NaN,9.00
3196,10317,6,2.0,6,6.0,NaN,NaN,9.00
3204,10409,6,2.0,6,7.0,NaN,NaN,10.50
3207,10439,6,2.0,6,6.0,NaN,NaN,9.00


In [333]:
whole_solution_df.query('n_couriers == 2 & employee_count == 2')

,area,period,n_couriers,day,deliveries,employee_count,employees_assign,cost_outsource
892,10247,7,2.0,1,8.0,2.0,"[1, 2]",NaN
1142,10439,3,2.0,2,8.0,2.0,"[4, 5]",NaN


In [334]:
theta = 2
area = 10999

a = area_map[area]
d = 0
r = region_area_map[area]
# couriers_needed[a, theta, d]
print(couriers_needed[a, theta, d])

# employees
for e in E[r]:
    val = k_var[e, a, theta, d].X
    if val > 0.5:
        print(e)

print(sum([k_var[e, a, theta, d].X for e in E[r]]))

# factor
factor = deliveries[a,theta,d] / couriers_needed[a,theta,d] * c_out
print(factor)

# outsource
print(omega_var[a, theta, d].X)

2.0
0.0
5.25
10.5


In [337]:
print(couriers_needed[a, theta, d] - sum([k_var[e, a, theta, d].X for e in E[r]]) * factor)
omega_var[a, theta, d].X

1.0


1.5

In [336]:
for r in R:
    for a in A[r]:
        for theta in Theta:
            for d in D:
                factor = deliveries[a,theta,d] * couriers_needed[a,theta,d] * c_out
                m.addConstr(
                    (couriers_needed[a, theta, d] - sum([k_var[e, a, theta, d] for e in E[r]]) * factor <= omega_var[a, theta, d]),
                    name = f'start_times_{r}_{e}_2'   
                )